    Atividade
        ● Selecione uma série temporal
        ● Gere as bases de treinamento e teste baseadas em janela de tempo
        ● Utilizando algum modelo de aprendizagem de máquinas, calcule os erros de previsão de longo alcance 
          (12 passos), pela abordagem iterativa e pela abordagem direta
        ○ Qual abordagem gerou os melhores resultados?

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [3]:
dados = pd.read_csv('CSVs/AirPassengers.csv', sep=',', header=0, index_col='Month')

In [4]:
#janela = series_to_supervised(data=dados, n_in=3, n_out=12)
#y = janela.iloc[:,[3]].values
#x = janela.iloc[:,:3].values
janela = series_to_supervised(data=dados, n_in=3, n_out=12)
y_iterativo = janela[['var1(t)']]
y_direto = janela[['var1(t+11)']]
x = janela[['var1(t-3)', 'var1(t-2)', 'var1(t-1)']]

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y_direto, test_size=0.3, random_state=0) # Direto - Y_direto

In [6]:
svr = SVR(kernel='linear', gamma=0.0001 , C=0.1)

In [7]:
svr.fit(x_train,y_train)

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.0001,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
predicao_direto = svr.predict(x_test)

In [9]:
print("MAE",mean_absolute_error(y_test,predicao_direto))

MAE 14.04155381088644


In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y_iterativo, test_size=0.3, random_state=0) # Iterativo - Y_iterativo

In [11]:
svr = SVR(kernel='linear', gamma=0.0001 , C=0.1)

In [12]:
svr.fit(x_train,y_train)

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.0001,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [13]:
predicao_iterativo = svr.predict(x_test)

In [14]:
print("MAE",mean_absolute_error(y_test,predicao_iterativo))

MAE 22.381692969027817
